In [1]:
#imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [2]:
#instanciando spark 
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Estatistica_descritiva') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

23/06/06 17:41:10 WARN Utils: Your hostname, caio-HP-250-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.122 instead (on interface wlo1)
23/06/06 17:41:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/06 17:41:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#diretorios de trabalho
fire_db = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/Fire_Department_Calls_for_Service/base_aulas/sf-fire-calls.csv'

In [4]:
#leitura da base de dados 

fire_df = (
    spark
    .read
    .csv(fire_db,header=True,inferSchema=True)
)

fire_df.show(truncate=False)


23/06/06 17:43:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+------------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                       |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType      |UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+-------

In [5]:
#contando registros
fire_df.count()

175296

In [6]:
fire_df.head(5)

[Row(CallNumber=20110016, UnitID='T13', IncidentNumber=2003235, CallType='Structure Fire', CallDate='01/11/2002', WatchDate='01/10/2002', CallFinalDisposition='Other', AvailableDtTm='01/11/2002 01:51:44 AM', Address='2000 Block of CALIFORNIA ST', City='SF', Zipcode=94109, Battalion='B04', StationArea='38', Box='3362', OriginalPriority='3', Priority='3', FinalPriority=3, ALSUnit=False, CallTypeGroup=None, NumAlarms=1, UnitType='TRUCK', UnitSequenceInCallDispatch=2, FirePreventionDistrict='4', SupervisorDistrict='5', Neighborhood='Pacific Heights', Location='(37.7895840679362, -122.428071912459)', RowID='020110016-T13', Delay=2.95),
 Row(CallNumber=20110022, UnitID='M17', IncidentNumber=2003241, CallType='Medical Incident', CallDate='01/11/2002', WatchDate='01/10/2002', CallFinalDisposition='Other', AvailableDtTm='01/11/2002 03:01:18 AM', Address='0 Block of SILVERVIEW DR', City='SF', Zipcode=94124, Battalion='B10', StationArea='42', Box='6495', OriginalPriority='3', Priority='3', FinalP

Trabalhando com SQL

In [7]:
fire_df.createOrReplaceTempView('fire_tb')

spark.sql('select * from fire_tb').show(truncate=False)

+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+------------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                       |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType      |UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+-------

In [10]:
spark.sql('select distinct(CallType) from fire_tb').show(truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

In [11]:
spark.sql('select CallType,count(*) as count from fire_tb \
          group by CallType order by count desc').show(truncate=False)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
|Odor (Strange / Unknown)       |490   |
|Electrical Hazard              |482   |
|Elevator / Escalator Rescue    |453   |
|Smoke Investigation (Outside)  |391   |
|Fuel Spill                     |193   |
|HazMat                         |124   |
|Industrial Accidents           |94    |
|Explosion                      |89    |
|Train / Rail Incident          |57    |
|Aircraft Emergency             |36    |
+-------------------------------+------+
only showing top

In [13]:
spark.sql('select CallNumber,Delay from fire_tb \
          where Delay > 50').show(truncate=False)

+----------+---------+
|CallNumber|Delay    |
+----------+---------+
|20150265  |95.28333 |
|20510101  |79.98333 |
|20750413  |336.33334|
|20750413  |330.66666|
|21730011  |59.65    |
|21780097  |54.45    |
|23110482  |106.05   |
|30330258  |65.28333 |
|30640355  |67.03333 |
|30740040  |316.58334|
|30800133  |628.61664|
|30910242  |131.93333|
|32170203  |115.48333|
|32520252  |56.033333|
|52780360  |53.083332|
|60690316  |99.63333 |
|60760172  |75.95    |
|82100052  |84.6     |
|82410212  |91.916664|
|82490156  |385.58334|
+----------+---------+
only showing top 20 rows

